# BlueField-2 SmartNICs
BlueField-2 is a next-generation Data Processing Unit (DPU) developed by NVIDIA (formerly Mellanox Technologies). It integrates powerful computing, networking, and storage acceleration capabilities into a single device. Designed for modern data centers, BlueField-2 DPUs offer hardware-accelerated data processing, efficient resource management, and enhanced security features.

## Key Features of BlueField-2
- Integrated Compute and Networking:
  - Combines an ARM-based SoC (System-on-Chip) with a ConnectX-6 Dx network interface controller (NIC).
  - Offers hardware acceleration for networking and storage operations.

- Advanced Networking:
  - Supports up to 200Gb/s Ethernet or InfiniBand networking.
  - Equipped with SR-IOV, RDMA, and GPUDirect Storage capabilities.
    
- Storage Acceleration:
  - Enables offloading for NVMe over Fabrics (NVMe-oF).
  - Provides support for RAID and data mirroring.

- Security Capabilities:
  - Includes a hardware root of trust and secure boot features.
  -  Offers real-time encryption, data isolation, and zero-trust security models.

## Create a slice using BlueField-2 SmartNICs 

This notebook shows how to create an isolated local Ethernet using BlueField2 Smart NICs and connect compute nodes to it and use FABlib's automatic configuration functionality.


## Import the FABlib Library


In [ ]:
from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network
import ipaddress

from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

fablib = fablib_manager()
                     
fablib.show_config();

## Create the Experiment Slice

The following creates two nodes with BlueField-2 NICs connected to an isolated local Ethernet.  

Two nodes are created and one NIC component is added to each node.  

This example uses components of model `NIC_BlueField2_ConnectX_6` and `NIC_Basic`. The components are passed to the node via PCI passthrough. Other NIC models are listed below. Calling the `get_interfaces()` method on a component will return a list of interfaces. Many dedicated NIC components may have more than one port. Either port can be connected to the network.

Automatic configuration requires specify a subnet for the network and setting the interface's mode to `auto` using the `iface1.set_mode('auto')` function before submitting the request. With automatic configuration, FABlib will allocate an IP from the network's subnet and configure the device during the post boot configuration stage.  Optionally, you can add routes to the node before submitting the request.


NIC component models options:
- NIC_Basic: 100 Gbps Mellanox ConnectX-6 SR-IOV VF (1 Port)
- NIC_ConnectX_5: 25 Gbps Dedicated Mellanox ConnectX-5 PCI Device (2 Ports) 
- NIC_ConnectX_6: 100 Gbps Dedicated Mellanox ConnectX-6 PCI Device (2 Ports)
- NIC_BlueField2_ConnectX_6: 100 Gbps Dedicated Mellanox BlueField-2 ConnectX-6 PCI Device (2 Ports) 

In [ ]:
slice_name = 'MySlice-bluefields'
#site = fablib.get_random_site()
site="RENC"
print(f"Site: {site}")

node1_name = 'Node1'
node2_name = 'Node2'

network_name='net1'
node1_image = "default_bluefield2_ubuntu_24"
node2_image = "default_ubuntu_24"

In [ ]:
#Create Slice
slice = fablib.new_slice(name=slice_name)

# Network
net1 = slice.add_l2network(name=network_name, subnet=IPv4Network("192.168.1.0/24"))

# Node1
node1 = slice.add_node(name=node1_name, site=site, image=node1_image)
iface1 = node1.add_component(model='NIC_BlueField2_ConnectX_6', name='nic1').get_interfaces()[0]
iface1.set_mode('auto')
net1.add_interface(iface1)

# Node2
node2 = slice.add_node(name=node2_name, site=site, image=node2_image)
iface2 = node2.add_component(model='NIC_Basic', name='nic1').get_interfaces()[0]
iface2.set_mode('auto')
net1.add_interface(iface2)


#Submit Slice Request
slice.submit();

## Configure the Bluefield Smart NIC

We configure the BlueField SmartNIC by assigning the private IP address `10.100.100.1` to the `tmfifo_net0` device, enabling communication and management of the BlueField-2 DPU. Additionally, we install the BlueField bundle (BFB) on the DPU via the specified RShim interface, updating its firmware and configuration to ensure optimal performance in data center environments.

You can force to configure using custom commands by passing list of commands string to `configure` function.

NOTE: When using `default_bluefield2_ubuntu_24` image for the Node i.e. `Node1` connected to BlueField-2 Smart NIC, the BFB image is available at the location: `/opt/bf-bundle/bf-bundle-2.9.1-40_24.11_ubuntu-22.04_prod.bfb`

In [ ]:
slice = fablib.get_slice(slice_name)

node1 = slice.get_node(name=node1_name) 
bluefield = node1.get_component(name='nic1')
output = bluefield.configure()

### Re-apply the network config post re-imaging

Reapply the network configuration on the Node connected to the BlueField. This is only effective when using automatic configuration, where interfaces are set up via `iface1.set_mode('auto')` or `iface1.set_mode('config')`.

In [ ]:
node1.config()
slice.list_interfaces();

## Run the Experiment

With automatic configuration the slice is ready for experimentation after it becomes active.  Note that automatic configuration works well when saving slices to a file and reinstantiating the slice.  Configuration tasks can be stored in the saved slice, reducing the complexity of notebooks and other runtime steps.

We will find the ping round trip time for this pair of sites.  Your experiment should be more interesting!

**Note:** If the ping fails, try rerunning the cell above to ensure the IP address is properly configured, then attempt the ping again.

In [ ]:
slice = fablib.get_slice(slice_name)

node1 = slice.get_node(name=node1_name)        
node2 = slice.get_node(name=node2_name)           

node2_addr = node2.get_interface(network_name=network_name).get_ip_addr()

stdout, stderr = node1.execute(f'ping -c 5 {node2_addr}')

## Delete the Slice

Please delete your slice when you are done with your experiment.

In [ ]:
slice = fablib.get_slice(slice_name)
slice.delete()